In [6]:
import pickle as pkl    
import os
import seaborn as sns
import matplotlib.pyplot as plt
from pcdet.ops.iou3d_nms import iou3d_nms_utils
import numpy as np
import tqdm
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns

import torch
os.chdir('/home/cgriesbacher/thesis/3DTrans/')

### Load AVL data

In [16]:
with open('data/avltruck/avl_infos_train.pkl', 'rb') as f:
    avl_infos = pkl.load(f)

frame_names = [info["point_cloud"]['lidar_idx'] for info in avl_infos]

categories = {}
name_example = 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000012.json',

for frame_name in frame_names:
    cat = frame_name.split('/')[1].split('_')[0]

    if cat not in categories:
        categories[cat] = 1
    else:
        categories[cat] += 1

categories

{'CityStreet': 7694,
 'CityThoroughfare': 6050,
 'MinorRoad': 1167,
 'Motorway': 9562,
 'PrimaryHighway': 4533,
 'SecondaryHighway': 2996}

In [20]:
with open('data/avlrooftop/avl_infos_train.pkl', 'rb') as f:
    avl_infos = pkl.load(f)

frame_names = [info["point_cloud"]['lidar_idx'] for info in avl_infos]

categories = {}
name_example = 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000012.json',

for frame_name in frame_names:
    cat1 = frame_name.split('/')[1].split('_')[0]
    cat2 = frame_name.split('/')[1].split('_')[1]
    cat3 = frame_name.split('/')[1].split('_')[2]
    if cat1 not in categories:
        categories[cat1] = {}
    if cat2 not in categories[cat1]:
        categories[cat1][cat2] = {}
    if cat3 not in categories[cat1][cat2]:
        categories[cat1][cat2][cat3] = 1
    else:
        categories[cat1][cat2][cat3] += 1


categories

#print gategorie matrix
for cat1 in categories:
    print(cat1)
    for cat2 in categories[cat1]:
        print('\t', cat2)
        for cat3 in categories[cat1][cat2]:
            print('\t\t', cat3, categories[cat1][cat2][cat3])

HIGHWAY
	 Normal
		 road 743
		 exit 240
		 entry 180
	 Sunny
		 entry 30
		 exit 60
		 road 90
	 Rain
		 road 536
		 entry 59
		 exit 60
CITY
	 Normal
		 roundabout 420
		 street 117
		 Tjunction 239
		 junction 418
	 Rain
		 Tjunction 120
		 street 120
		 junction 150
		 roundabout 60
	 Sunny
		 junction 56
		 street 180
		 roundabout 30
INTERURBAN
	 Normal
		 road 441
		 Tjunction 326
		 roundabout 389
		 junction 201
	 Sunny
		 road 145
		 junction 60
		 roundabout 30
	 Rain
		 road 146
		 Tjunction 30
		 roundabout 121
		 junction 30


In [18]:
frame_name

'sequences/INTERURBAN_Normal_road_20200505110758/unpacked/lidar/0030.pkl'

In [13]:
frame_names

['sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000008.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000013.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000012.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000007.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000016.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000018.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000005.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000001.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000017.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000010.json',
 'sequences/CityStreet_dgt_2021-11-26-13-36-42_0_s0/dataset/logical_frame_000002.json',
 'sequences/CityStreet_dgt_2021-

In [15]:
categories

{'CityStreet': 7694,
 'CityThoroughfare': 6050,
 'MinorRoad': 1167,
 'Motorway': 9562,
 'PrimaryHighway': 4533,
 'SecondaryHighway': 2996}

### Load ZOD data

In [8]:
from zod import ZodFrames
from zod.constants import Camera

#load zod
root_dir = '../../data/zod/'
frames = ZodFrames(root_dir, "full")

FileNotFoundError: Could not find trainval file: ../../data/zod/trainval-frames-full.json.
Make sure you have downloaded ZodFrames-full to ../../data/zod/